<a href="https://colab.research.google.com/github/mspendyala/aws_bedrock_samples/blob/main/aws_bedrock_example_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install boto3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 7.3 MB/s eta 0:00:00


In [ ]:
import os

# Replace with your AWS credentials (use temporary credentials if possible)
os.environ["AWS_ACCESS_KEY_ID"] = ""
os.environ["AWS_SECRET_ACCESS_KEY"] = ""
os.environ["AWS_REGION"] = "us-east-1"

print("AWS credentials configured securely in Colab!")

In [ ]:
import boto3
import json
import os
from botocore.exceptions import NoCredentialsError, ClientError

# Load AWS credentials from environment variables (Google Colab or local)
aws_access_key = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
aws_session_token = os.getenv("AWS_SESSION_TOKEN")
aws_region = os.getenv("AWS_REGION", "us-east-1")

# Initialize AWS Clients
s3_client = boto3.client("s3", region_name=aws_region)
bedrock_client = boto3.client("bedrock-runtime", region_name=aws_region)

# S3 Bucket and File
S3_BUCKET = "my-travel-knowledge"
S3_FILE_KEY = "travel_guide.json"

def check_s3_file():
    """Check if the knowledge base file exists in S3"""
    try:
        response = s3_client.list_objects_v2(Bucket=S3_BUCKET)
        if "Contents" in response:
            for obj in response["Contents"]:
                if obj["Key"] == S3_FILE_KEY:
                    return True
        print(f"⚠️ File {S3_FILE_KEY} NOT found in S3 bucket `{S3_BUCKET}`.")
        return False
    except ClientError as e:
        print(f"❌ Error checking S3: {e}")
        return False

def fetch_knowledge_from_s3():
    """Retrieve travel data from S3"""
    try:
        response = s3_client.get_object(Bucket=S3_BUCKET, Key=S3_FILE_KEY)
        travel_data = json.loads(response["Body"].read().decode("utf-8"))
        return travel_data
    except NoCredentialsError:
        print("❌ AWS credentials not found!")
        return []
    except ClientError as e:
        print(f"❌ Error fetching knowledge base: {e}")
        return []

def retrieve_travel_info(destination, travel_guide):
    """Retrieve relevant travel information from S3 knowledge base"""
    for entry in travel_guide:
        if destination.lower() in entry["destination"].lower():
            return entry  # Return matching travel info
    return None  # No relevant info found

def format_travel_info(travel_info):
    """Format travel information for AI model"""
    return f"""
    **Destination:** {travel_info['destination']}
    **Top Attractions:** {', '.join(travel_info['attractions'])}
    **Best Time to Visit:** {travel_info['best_time']}
    **Local Food to Try:** {', '.join(travel_info['local_food'])}
    **Travel Tips:** {travel_info['tips']}
    """

def query_bedrock(prompt):
    """Query Amazon Bedrock with the correct prompt format for Claude"""
    formatted_prompt = f'\n\nHuman: {prompt}\n\nAssistant:'  # 👈 Add 'Human:' prefix

    try:
        response = bedrock_client.invoke_model(
            modelId="anthropic.claude-instant-v1",  # Change model if needed
            body=json.dumps({
                "prompt": formatted_prompt,
                "max_tokens_to_sample": 250  # Adjust token limit
            })
        )
        result = json.loads(response["body"].read().decode("utf-8"))
        return result.get("completion", "No response from model")
    except ClientError as e:
        print(f"❌ Error calling Bedrock: {e}")
        return "Error in Bedrock call."

def main():
    """Main workflow for Travel Guide Chatbot"""
    user_query = "Tell me the best places to visit in Paris."
    destination = "Paris"  # Extracted from query (can be automated later)

    # Step 1: Check if the S3 knowledge base exists
    if not check_s3_file():
        return

    # Step 2: Fetch travel data from S3
    travel_guide = fetch_knowledge_from_s3()

    # Step 3: Retrieve relevant travel information
    travel_info = retrieve_travel_info(destination, travel_guide)

    if travel_info:
        retrieved_context = format_travel_info(travel_info)
    else:
        retrieved_context = "No relevant travel information found."

    # Step 4: Create prompts for Bedrock
    vanilla_prompt = f"Tell me the best places to visit in {destination}."
    rag_prompt = f"Using the following travel information:\n{retrieved_context}\n\nAnswer this: What are the best places to visit in {destination}?"

    # Step 5: Query Bedrock
    response_vanilla = query_bedrock(vanilla_prompt)
    response_rag = query_bedrock(rag_prompt)

    # Step 6: Display results
    print("\n🔹 **Vanilla Bedrock Response:**")
    print(response_vanilla)
    print("\n🔹 **RAG-Enhanced Response (Using S3):**")
    print(response_rag)

if __name__ == "__main__":
    main()



🔹 **Vanilla Bedrock Response:**
 Here are some of the top places to visit in Paris:

- Eiffel Tower - The iconic iron lattice tower is one of the most recognizable structures in the world. Be sure to book tickets in advance.

- Notre Dame Cathedral - This Gothic cathedral is famous for its rose windows, flying buttresses and gargoyle statues. 

- Louvre Museum - Home to one of the largest art collections in the world, including the Mona Lisa. Allow at least half a day to see some of the highlights.

- Arc de Triomphe - This iconic arch stands at the western end of the Champs-Elysées. You can glimpse down the grand boulevard from the top.

- Palace of Versailles - A opulent palace and estate just outside Paris that was once home to French monarchs. Plan at least a full day trip.

- Champs-Elysées - Paris' most famous avenue lined with stores, cinemas, restaurants and gardens. Great for people watching.

- Jardin du Luxembourg - These beautiful French gardens are a lovely place to relax